API and script test with just the first listing data

In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
import numpy as np
import os

def get_market_value(row):
    # Get API key from environment variable
    API_KEY = os.getenv('MARKETCHECK_API_KEY')
    if not API_KEY:
        raise ValueError("MARKETCHECK_API_KEY not found in environment variables")

    base_url = "http://api.marketcheck.com/v2/search/car/active"
       
    # Convert mileage string to number
    mileage = int(''.join(filter(str.isdigit, str(row['clean_mileage']))))
    
    print(f"\nMaking API request for:")
    print(f"Year: {row['year']}")
    print(f"Make: {row['make']}")
    print(f"Model: {row['model']}")
    print(f"Trim: {row['trim']}")
    print(f"Mileage: {mileage}")
    
    # Parameters for the search
    params = {
        'api_key': API_KEY,
        'year': row['year'],
        'make': row['make'],
        'model': row['model'],
        'trim': row['trim'],
        'miles_range': f'{mileage-10000}-{mileage+10000}',
        'latitude': 40.7608,  # Salt Lake City coordinates
        'longitude': -111.8910,
        'radius': 500,
        'facets': 'price',
        'stats': 'price,miles',
        'dealer_type': 'All'
    }
    
    try:
        print("\nSending API request...")
        response = requests.get(base_url, params=params)
        print(f"Response status code: {response.status_code}")
        
        data = response.json()
        
        if response.status_code == 200:
            stats = data.get('stats', {})
            price_stats = stats.get('price', {})
            
            result = {
                'mean_price': price_stats.get('mean', None),
                'median_price': price_stats.get('median', None),
                'min_price': price_stats.get('min', None),
                'max_price': price_stats.get('max', None),
                'std_dev': price_stats.get('std', None),
                'sample_size': stats.get('count', 0)
            }
            
            print("\nReceived market values:")
            for key, value in result.items():
                print(f"{key}: {value}")
                
            return result
        else:
            print(f"\nError response: {data}")
            return None
            
    except Exception as e:
        print(f"\nException occurred: {e}")
        return None

# Load the data
print("Loading CSV file...")
df = pd.read_csv('ksl_cars_detailed.csv')
print(f"Total rows in dataset: {len(df)}")

# Clean price and mileage columns
def clean_price(price):
    if isinstance(price, str):
        return float(price.replace('$', '').replace(',', ''))
    return price

def clean_mileage(mileage):
    if isinstance(mileage, str):
        return float(mileage.replace(' Miles', '').replace(',', ''))
    return mileage

# Process only the first row
first_row = df.iloc[0].copy()
print("\nProcessing first row:")
print(first_row[['year', 'make', 'model', 'trim', 'mileage', 'price']])

# Clean the price and mileage for the first row
first_row['clean_price'] = clean_price(first_row['price'])
first_row['clean_mileage'] = clean_mileage(first_row['mileage'])

# Get market value for the first row
print("\nGetting market value...")
market_value = get_market_value(first_row)

if market_value:
    # Calculate price differences
    price_diff = first_row['clean_price'] - market_value['median_price']
    price_diff_pct = (price_diff / market_value['median_price']) * 100
    
    print("\nAnalysis Results:")
    print("-" * 50)
    print(f"Listing Price: ${first_row['clean_price']:,.2f}")
    print(f"Market Median: ${market_value['median_price']:,.2f}")
    print(f"Price Difference: ${price_diff:,.2f}")
    print(f"Price Difference %: {price_diff_pct:.1f}%")
    
    # Determine deal rating
    if price_diff_pct <= -20:
        deal_rating = "Excellent Deal"
    elif price_diff_pct <= -10:
        deal_rating = "Good Deal"
    elif price_diff_pct <= 0:
        deal_rating = "Fair Price"
    elif price_diff_pct <= 10:
        deal_rating = "Slightly High"
    elif price_diff_pct <= 20:
        deal_rating = "High"
    else:
        deal_rating = "Very High"
    
    print(f"Deal Rating: {deal_rating}")
    print(f"Sample Size: {market_value['sample_size']} comparable vehicles")
else:
    print("\nNo market value data available for this vehicle")

/Users/chandlerward/car-scraper/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Loading CSV file...
Total rows in dataset: 24

Processing first row:
year                2017
make               Buick
model            Cascada
trim       Sport Touring
mileage     99,749 Miles
price            $12,997
Name: 0, dtype: object

Getting market value...


ValueError: MARKETCHECK_API_KEY not found in environment variables

This is the test for the entire .csv data set - not revised

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime
import numpy as np
import os

def get_market_value(row):
    # Get API key from environment variable
    API_KEY = os.getenv('MARKETCHECK_API_KEY')
    if not API_KEY:
        raise ValueError("MARKETCHECK_API_KEY not found in environment variables")

    base_url = "http://api.marketcheck.com/v2/search/car/active"

    # Convert mileage string to number
    mileage = int(''.join(filter(str.isdigit, str(row['clean_mileage']))))
    
    # Parameters for the search
    params = {
        'api_key': API_KEY,
        'year': row['year'],
        'make': row['make'],
        'model': row['model'],
        'trim': row['trim'],
        'miles_range': f'{mileage-10000}-{mileage+10000}',  # Look for similar mileage
        'latitude': 40.7608,  # Salt Lake City coordinates
        'longitude': -111.8910,
        'radius': 500,  # Search radius in miles
        'facets': 'price',  # Request price aggregations
        'stats': 'price,miles',  # Request statistical data
        'dealer_type': 'All'  # Include both dealers and private sellers
    }
    
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        
        if response.status_code == 200:
            stats = data.get('stats', {})
            price_stats = stats.get('price', {})
            
            return {
                'mean_price': price_stats.get('mean', None),
                'median_price': price_stats.get('median', None),
                'min_price': price_stats.get('min', None),
                'max_price': price_stats.get('max', None),
                'std_dev': price_stats.get('std', None),
                'sample_size': stats.get('count', 0)
            }
        else:
            print(f"Error for {row['year']} {row['make']} {row['model']}: {data.get('error', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"Exception for {row['year']} {row['make']} {row['model']}: {e}")
        return None

# Load and clean the data
df = pd.read_csv('ksl_cars_detailed.csv')

# Clean price and mileage columns
def clean_price(price):
    if isinstance(price, str):
        return float(price.replace('$', '').replace(',', ''))
    return price

def clean_mileage(mileage):
    if isinstance(mileage, str):
        return float(mileage.replace(' Miles', '').replace(',', ''))
    return mileage

df['clean_price'] = df['price'].apply(clean_price)
df['clean_mileage'] = df['mileage'].apply(clean_mileage)

# Get market values for each listing
market_values = []
for idx, row in df.iterrows():
    print(f"Processing {row['year']} {row['make']} {row['model']}...")
    market_value = get_market_value(row)
    market_values.append(market_value)

# Convert market values to DataFrame columns
market_values_df = pd.DataFrame(market_values)
df = pd.concat([df, market_values_df], axis=1)

# Calculate price differences
df['price_difference'] = df['clean_price'] - df['median_price']
df['price_difference_percentage'] = (df['price_difference'] / df['median_price']) * 100

# Add deal rating
df['deal_rating'] = pd.cut(
    df['price_difference_percentage'],
    bins=[-np.inf, -20, -10, 0, 10, 20, np.inf],
    labels=['Excellent Deal', 'Good Deal', 'Fair Price', 'Slightly High', 'High', 'Very High']
)

# Save results
df.to_csv('ksl_cars_analysis.csv', index=False)

# Print summary
print("\nAnalysis Summary:")
print("-" * 50)
print(f"Total listings analyzed: {len(df)}")
print("\nDeal Distribution:")
print(df['deal_rating'].value_counts())
print("\nTop 5 Best Deals:")
print(df.sort_values('price_difference_percentage').head()[
    ['year', 'make', 'model', 'clean_price', 'median_price', 
     'price_difference', 'price_difference_percentage', 'deal_rating', 'url']
])